<a href="https://colab.research.google.com/github/siddadel/Deep-Fashion-Python/blob/master/TrustLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install warcio
# !pip install langdetect

In [2]:
from pprint import pprint
import gzip
from urllib.request import urlopen
from bs4 import BeautifulSoup
from langdetect import detect
import threading
from warcio.archiveiterator import ArchiveIterator
import re
import requests
import sys
import calendar
import re
import datetime

In [3]:
#presently we support only english
#Potential for BIAS!
valid_locale = ["en"]
valid_tlds = [".com/"]
valid_domains = ["https://www.cnn.com/"]

main_regex = re.compile("(covid|pandemic|covid-19)", re.IGNORECASE)
aux_regex = re.compile("(economy|cost|inflation|loss|economic|costs)", re.IGNORECASE)

def find_relevant(contents, uri, month, id):
  soup = BeautifulSoup(contents, 'html5lib')
  [s.decompose() for s in soup("script")]  # remove <script> elements
  body_text = soup.body.get_text().lower()
  if len(body_text)!=0 and is_lang_valid(body_text):
      m1 = main_regex.search(body_text)
      if(m1):
        m2 = aux_regex.search(body_text)
        if(m2):
          print(("%s,%s")%(calendar.month_abbr[month], uri))

def is_tld_valid(uri):
  for tld in valid_tlds:
    if tld in uri:
      return True
  return False

def is_valid_domain(uri):
  return True
  # for domain in valid_domains:
  #   if domain in uri:
  #     return True
  # return False

def is_lang_valid(body_text):
  lang = detect(body_text)
  return lang in valid_locale

def get_valid_uri(record):
  uri = record.rec_headers.get_header("WARC-Target-URI")
  if record.rec_type != "warcinfo" and is_tld_valid(uri) and is_valid_domain(uri):
    # pprint(vars(record))
    return uri
  else:
    return None

def parse_segment(file_name):
  try:
    id = 0
    stream = requests.get(file_name, stream=True).raw
    for record in ArchiveIterator(stream):
      tz = record.rec_headers.get_header("WARC-Date")
      # date_time_obj = datetime.strptime(tz, '%y-%m-%dT%H:%M:%SZ')
      tokens = tz.split('-')
      uri = get_valid_uri(record)
      if uri:
        contents = (
              record.content_stream()
              .read()
              .decode("utf-8", "replace")
          )
        find_relevant(contents, uri,int(tokens[1]), id)
        id+=1
  except Exception as e:
     pass
    # print("{} for {}".format(e, uri))

In [ ]:
i_s = [5, 10, 16, 24, 29, 34, 40, 45, 50]
i_s.reverse()
for i in i_s:
  file_name = "https://data.commoncrawl.org/crawl-data/CC-MAIN-2020-{0:0=2d}/warc.paths.gz".format(i)
  stream = requests.get(file_name, stream=True).raw
  with gzip.open(stream, 'rt') as f:
    file_content = f.read()
    threads = []
    for line in file_content.splitlines():
        link = "http://commoncrawl.s3.amazonaws.com/{}".format(line)
        thread = threading.Thread(target=parse_segment, args=(link,), daemon=True)
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()  